<a href="https://colab.research.google.com/github/sssylvia0/Uplift_Model/blob/main/Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

結果資料整理

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [33]:
usecols = ['UNINO', '標籤', 'TAG', '發送名單']
df = pd.read_csv('/content/drive/MyDrive/Sinopac/11月外匯名單_成效追蹤.csv', usecols=usecols)
df

,UNINO,標籤,TAG,發送名單
0,A1220046320,NaN,數位通路_換匯名單_F群_未發,0
1,A1221949610,數位通路_換匯名單_F群,NaN,1
2,A1224193120,數位通路_換匯名單_G群,NaN,1
3,A1228308980,數位通路_換匯名單_G群,NaN,1
4,A1228865070,數位通路_換匯名單_F群,NaN,1
...,...,...,...,...
105268,Y2725958190,數位通路_換匯名單_C群,NaN,1
105269,Y2727395520,NaN,數位通路_換匯名單_F群_未發,0
105270,Y2727650750,數位通路_換匯名單_F群,NaN,0
105271,Z1222922330,數位通路_換匯名單_G群,NaN,0


In [145]:
import pandas as pd
import numpy as np

df_new = df[['UNINO']]

# 合併兩個欄位，並新增一個欄位標示哪個曾經有NaN
df_new['標籤'] = df['標籤'].fillna(df['TAG'])
df_new['Planned T'] = df['TAG'].isnull().astype(int)
df_new['Actual T'] = df['發送名單']
df_new.drop(df_new[df_new['標籤'].isnull()].index, inplace=True)

df_new.reset_index(drop=True, inplace=True)
df_new

<ipython-input-145-ed3657d90190>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['標籤'] = df['標籤'].fillna(df['TAG'])
<ipython-input-145-ed3657d90190>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Planned T'] = df['TAG'].isnull().astype(int)


,UNINO,標籤,Planned T,Actual T
0,A1220046320,數位通路_換匯名單_F群_未發,0,0
1,A1221949610,數位通路_換匯名單_F群,1,1
2,A1224193120,數位通路_換匯名單_G群,1,1
3,A1228308980,數位通路_換匯名單_G群,1,1
4,A1228865070,數位通路_換匯名單_F群,1,1
...,...,...,...,...
105046,Y2725958190,數位通路_換匯名單_C群,1,1
105047,Y2727395520,數位通路_換匯名單_F群_未發,0,0
105048,Y2727650750,數位通路_換匯名單_F群,1,0
105049,Z1222922330,數位通路_換匯名單_G群,1,0


In [146]:
#標籤, TAG轉換
# 定義字母到二進制編碼的映射
binary_mapping = {
    'A': '100',
    'B': '010',
    'C': '001',
    'D': '110',
    'E': '011',
    'F': '101',
    'G': '111'
}

# Define the function to convert labels to binary representation
# 定義函數來將標籤轉換成二進制編碼
def convert_to_binary(label):
    binary_label = ''
    # 檢查標籤中的每個字母
    for char in label:
        # 如果字母是英文字母且在映射中存在，則將其對應的二進制編碼加入到 binary_label 中
        if char in binary_mapping:
            binary_label += binary_mapping[char.upper()]
    return binary_label


binary_label = df_new['標籤'].apply(convert_to_binary)
df_new['標籤'] = binary_label
# 将 '標籤' 列拆分成三列
df_new[['S', 'L', 'V']] = df_new['標籤'].str.split('', expand=True).iloc[:, 1:4]
# 移除原始的 '標籤' 列
df_new.drop('標籤', axis=1, inplace=True)
df_new['Round'] = 1
df_new

,UNINO,Planned T,Actual T,S,L,V,Round
0,A1220046320,0,0,1,0,1,1
1,A1221949610,1,1,1,0,1,1
2,A1224193120,1,1,1,1,1,1
3,A1228308980,1,1,1,1,1,1
4,A1228865070,1,1,1,0,1,1
...,...,...,...,...,...,...,...
105046,Y2725958190,1,1,0,0,1,1
105047,Y2727395520,0,0,1,0,1,1
105048,Y2727650750,1,0,1,0,1,1
105049,Z1222922330,1,0,1,1,1,1


In [147]:
#Prediction results
usecols = ['UNINO', 'Uplift_mean|T = 0', 'Uplift_mean|T = 1', 'X_te_2', 'E(V)|X,T = 0', 'E(V)|X,T = 1', 'Value_X']
res = pd.read_csv('/content/drive/MyDrive/Sinopac/Results/Prediction_results_All_202308_new.csv', usecols = usecols)
res = pd.merge(df_new.UNINO, res, on='UNINO', how='left')
res.columns = ['UNINO', 'Lift|T = 0', 'Lift|T = 1', 'Lift', 'Value|T=0', 'Value|T=1', 'Value']
res

,UNINO,Lift|T = 0,Lift|T = 1,Lift,Value|T=0,Value|T=1,Value
0,A1220046320,0.004380,0.034457,0.014286,761.894511,4448.653440,1169.035823
1,A1221949610,0.005012,0.030657,0.010935,798.304563,4460.122564,712.987875
2,A1224193120,0.005652,0.035673,0.038840,801.189474,4401.355675,670.351594
3,A1228308980,0.006020,0.036407,0.036564,1547.620644,7787.359599,33852.645364
4,A1228865070,0.004124,0.036939,0.008140,689.024418,5509.028624,625.983806
...,...,...,...,...,...,...,...
105046,Y2725958190,0.007029,0.031847,0.004010,1656.168757,6539.039792,5027.934584
105047,Y2727395520,0.005046,0.030175,0.007685,686.915001,3642.971152,896.318976
105048,Y2727650750,0.005525,0.036715,0.008507,597.599400,3363.202331,1121.475990
105049,Z1222922330,0.005011,0.035214,0.035122,591.849539,3213.288381,974.855341


In [ ]:
import zipfile

# Specify the path to the zip file
zip_file_path = '/content/drive/MyDrive/Sinopac/Rawdata/data202308_v1.csv.zip'

# Unzip the folder
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')
print("Unzipped successfully.")

In [140]:
#Feature X
X = pd.read_csv('/content/data202308_v1.csv', index_col=0)
#X[['換匯發送名單','TOTALNTAMOUNT', '跨售行為']]
X = pd.merge(df_new.UNINO, X, on='UNINO', how='left')
X

,UNINO,年齡,公司戶記號,性別,呆帳記號_含信用卡,客戶年收入,網路會員記號,網路使用記號_行動銀行,小mma記號,網路服務記號,...,客戶行業別_醫療保健,客戶行業別_金融業,客戶行業別_電子業,對帳單寄送記號_0,對帳單寄送記號_1,對帳單寄送記號_2,信用卡VIP客戶註記_Y,外幣存款記號_Y,信用卡轉呆記號_Y,信用卡自動扣款記號_Y
0,A1220046320,73,0,1.0,0,0.0,1,1,0,1,...,0,0,0,0,0,1,0,0,0,0
1,A1221949610,75,0,1.0,0,0.0,1,2,0,1,...,0,0,0,1,0,0,0,0,0,1
2,A1224193120,71,0,1.0,0,1500.0,1,2,0,1,...,0,0,0,0,0,1,0,1,0,1
3,A1228308980,87,0,1.0,0,4500.0,1,2,0,1,...,0,0,0,0,0,1,0,1,0,1
4,A1228865070,73,0,1.0,0,0.0,1,2,0,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105046,Y2725958190,56,0,2.0,0,1000.0,1,2,0,3,...,0,0,0,1,0,0,0,1,0,1
105047,Y2727395520,64,0,2.0,0,120.0,1,2,0,3,...,0,0,0,1,0,0,0,1,0,1
105048,Y2727650750,51,0,2.0,0,648.0,1,2,0,1,...,0,0,0,0,0,1,0,1,0,1
105049,Z1222922330,46,0,1.0,0,641.0,1,2,0,1,...,0,0,0,0,0,1,0,1,0,0


In [161]:
#CONCAT
#UNINO, Round, S, L, V, Actual T, Planned T, X, Lift|T=0, Lift|T=1, Value|T=0, Value|T=1, TOTALNTAMOUNT
Concat = df_new[['UNINO', 'Round', 'S', 'L', 'V', 'Actual T', 'Planned T']]
Concat = pd.concat([Concat, X.drop('UNINO', axis=1)], axis = 1)
Concat = pd.concat([Concat, res.drop('UNINO', axis=1)], axis = 1)
Concat

,UNINO,Round,S,L,V,Actual T,Planned T,年齡,公司戶記號,性別,...,信用卡VIP客戶註記_Y,外幣存款記號_Y,信用卡轉呆記號_Y,信用卡自動扣款記號_Y,Lift|T = 0,Lift|T = 1,Lift,Value|T=0,Value|T=1,Value
0,A1220046320,1,1,0,1,0,0,73,0,1.0,...,0,0,0,0,0.004380,0.034457,0.014286,761.894511,4448.653440,1169.035823
1,A1221949610,1,1,0,1,1,1,75,0,1.0,...,0,0,0,1,0.005012,0.030657,0.010935,798.304563,4460.122564,712.987875
2,A1224193120,1,1,1,1,1,1,71,0,1.0,...,0,1,0,1,0.005652,0.035673,0.038840,801.189474,4401.355675,670.351594
3,A1228308980,1,1,1,1,1,1,87,0,1.0,...,0,1,0,1,0.006020,0.036407,0.036564,1547.620644,7787.359599,33852.645364
4,A1228865070,1,1,0,1,1,1,73,0,1.0,...,0,0,0,0,0.004124,0.036939,0.008140,689.024418,5509.028624,625.983806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105046,Y2725958190,1,0,0,1,1,1,56,0,2.0,...,0,1,0,1,0.007029,0.031847,0.004010,1656.168757,6539.039792,5027.934584
105047,Y2727395520,1,1,0,1,0,0,64,0,2.0,...,0,1,0,1,0.005046,0.030175,0.007685,686.915001,3642.971152,896.318976
105048,Y2727650750,1,1,0,1,0,1,51,0,2.0,...,0,1,0,1,0.005525,0.036715,0.008507,597.599400,3363.202331,1121.475990
105049,Z1222922330,1,1,1,1,0,1,46,0,1.0,...,0,1,0,0,0.005011,0.035214,0.035122,591.849539,3213.288381,974.855341


In [ ]:
#儲存結果
#Round1 105051 還沒補TOTALNTAMOUNT
#Round2